In [1]:
%%time

import malaya

CPU times: user 4.36 s, sys: 840 ms, total: 5.2 s
Wall time: 4.77 s


## Why augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! We provided few augmentation interfaces in Malaya.

## Synonym

Use dictionary of synonym to replace words with it synonyms. Synonym data from [Malaya-Dataset/90k-synonym](https://github.com/huseinzol05/Malaya-Dataset#90k-synonym).

```python
def synonym(
    string: str,
    threshold: float = 0.5,
    top_n = 5,
    cleaning_function: Callable = augmentation_textcleaning,
    **kwargs
):
    """
    augmenting a string using synonym, https://github.com/huseinzol05/Malaya-Dataset#90k-synonym

    Parameters
    ----------
    string: str
    threshold: float, optional (default=0.5)
        random selection for a word.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.
    cleaning_function: function, (default=malaya.text.function.augmentation_textcleaning)
        function to clean text.

    Returns
    -------
    result: List[str]
    """
```

In [2]:
string = 'saya suka makan ayam dan ikan'
text = 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.'

In [3]:
malaya.augmentation.synonym(string)

['saya suka makan ayam dan ikan',
 'saya mencinta makan ayam jantan dan ikan',
 'saya mencinta makan makan ayam jantan dan ikan',
 'saya suka makan makan ayam dan ikan',
 'saya suka makan ayam dan ikan']

In [4]:
malaya.augmentation.synonym(text)

['Perdana menteri menunjukkan beliau perlu memperoleh maklumat Terperinci menghayati isu berkenaan sebelum kerajaan dapat mengawali sebarang nombor lanjut bagaimanapun beliau beramanah sedih itu dapat diselesaikan dan pengurusannya kerajaan berupaya berfungsi dengan baik',
 'Perdana menteri menunjukkan beliau wajib mengusahakan data Terperinci menghayati penerbitan berkenaan di hadapan jajahan menggunakannya mendapatkan sebarang digit tertua bagaimanapun beliau beramanah suram itu dapatkan diselesaikan dan pengurusannya kabinet boleh mengangkut dengan baik',
 'Ulung uskup merupakan beliau wajib mengusahakan data Terperinci mempunyai penerbitan berkenaan di hadapan kerajaan menggunakannya berkumpul sebarang nombor gelap bagaimanapun beliau beramanah daif itu memperoleh diselesaikan dan pengurusannya kerajaan boleh mencari dengan baik',
 'Ulung menteri merupakan beliau wajib memupuk dokumen Terperinci mempunyai pengeluaran berkenaan sebelum kerajaan menangani berkumpul sebarang nombor ge

## Wordvector

dictionary of synonym is quite hard to populate, required some domain experts to help us. So we can use wordvector to find nearest words.

```python
def wordvector(
    string: str,
    wordvector,
    threshold: float = 0.5,
    top_n: int = 5,
    soft: bool = False,
    cleaning_function: Callable = augmentation_textcleaning,
):
    """
    augmenting a string using wordvector.

    Parameters
    ----------
    string: str
    wordvector: object
        wordvector interface object.
    threshold: float, optional (default=0.5)
        random selection for a word.
    soft: bool, optional (default=False)
        if True, a word not in the dictionary will be replaced with nearest jarowrinkler ratio.
        if False, it will throw an exception if a word not in the dictionary.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.
    cleaning_function: function, (default=malaya.text.function.augmentation_textcleaning)
        function to clean text.

    Returns
    -------
    result: List[str]
    """
```

In [5]:
vocab_wiki, embedded_wiki = malaya.wordvector.load_wiki()
word_vector_wiki = malaya.wordvector.load(embedded_wiki, vocab_wiki)

In [6]:
malaya.augmentation.wordvector(
    string, word_vector_wiki, soft = True
)

['saya suka makan ayam dan ikan',
 'kamu gemar minum ayam serta ayam',
 'anda pandai tidur ayam atau ular',
 'kami senang mandi ayam mahupun keju',
 'aku ingin berehat ayam tetapi lembu']

In [7]:
malaya.augmentation.wordvector(
    text, word_vector_wiki, soft = True
)

['perdana menteri berkata beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut bagaimanapun beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik',
 'perdana kementerian menyatakan beliau perlu memperoleh maklumat terperinci berkaitan persoalan berkaitan selepas kerajaan dapat mendapat sebarang tindakan terperinci walaupun dia sedar gangguan itu boleh dibuktikan serta pentadbiran kerajaan dapat dikelaskan dengan baik',
 'perdana setiausaha mengatakan beliau perlu memperoleh maklumat terperinci berhadapan prosedur tertentu setelah kerajaan dapat menghabiskan sebarang tindakan lanjutan namun baginda bimbang kelemahan itu harus dilaksanakan atau pentadbiran kerajaan harus bertindak dengan baik',
 'perdana jabatan mendapati beliau perlu memperoleh maklumat terperinci sejajar artikel tersebut ketika kerajaan dapat mengubah sebarang tindakan ringkas maka mereka menyangka gejala

## Transformer

Problem with wordvector, it just replaced a word for near synonym without understood the whole sentence context, so, Transformer comes to the rescue!

```python
def transformer(
    string: str,
    model,
    threshold: float = 0.5,
    top_p: float = 0.9,
    top_k: int = 100,
    temperature: float = 1.0,
    top_n: int = 5,
    cleaning_function: Callable = None,
):

    """
    augmenting a string using transformer + nucleus sampling / top-k sampling.

    Parameters
    ----------
    string: str
    model: object
        transformer interface object. Right now only supported BERT, ALBERT and ELECTRA.
    threshold: float, optional (default=0.5)
        random selection for a word.
    top_p: float, optional (default=0.8)
        cumulative sum of probabilities to sample a word. 
        If top_n bigger than 0, the model will use nucleus sampling, else top-k sampling.
    top_k: int, optional (default=100)
        k for top-k sampling.
    temperature: float, optional (default=0.8)
        logits * temperature.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.
    cleaning_function: function, (default=None)
        function to clean text.

    Returns
    -------
    result: List[str]
    """
```

In [8]:
electra = malaya.transformer.load(model = 'electra')

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.





INFO:tensorflow:Restoring parameters from /Users/huseinzolkepli/Malaya/electra-model/base/electra-base/model.ckpt


In [10]:
malaya.augmentation.transformer(text, electra)

['Perdana Menteri berkata , kerajaan sudah memperoleh maklumat terperinci berhubung masalah berkenaan supaya kerajaan dapat mengambil pelbagai tindakan sewajarnya . Bagaimanapun , beliau yakin masalah itu berjaya diselesaikan dan akhirnya terdahulu boleh diselesaikan dengan baik .',
 'Perdana Menteri berkata , kerajaan perlu memperoleh maklumat terperinci berhubung isu berkenaan supaya kerajaan dapat mengambil serius tindakan segera . Bagaimanapun , beliau berharap masalah itu boleh diselesaikan dan akhirnya kementerian boleh diselesaikan dengan baik .',
 'Perdana Menteri berkata , kerajaan telah memperoleh maklumat terperinci berhubung isu berkenaan supaya kerajaan dapat mengambil beberapa tindakan sewajarnya . Bagaimanapun , beliau berharap masalah itu perlu diselesaikan dan siasatan BN boleh diselesaikan dengan baik .',
 'Perdana Menteri berkata , kerajaan akan memperoleh maklumat terperinci berhubung isu berkenaan supaya kerajaan dapat mengambil sebarang tindakan susulan . Bagaiman